In [2]:
import pandas as pd
import numpy as np

#### **Creating the "receita_geral_RS.csv"**

In [3]:
files = [
    'receita_anual_2017_RS.csv',
    'receita_anual_2018_RS.csv',
    'receita_anual_2019_RS.csv',
    'receita_anual_2020_RS.csv',
    'receita_anual_2021_RS.csv',
    'receita_anual_2022_RS.csv',
    'receita_anual_2023_RS.csv',
    'receita_anual_2024_RS.csv',
]

years = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

tables = []
for file, year in zip(files, years):
    data = pd.read_csv(file, encoding = 'latin1', sep = ';')
    data['AA_EXERCICIO'] = year
    tables.append(data)

collection = pd.concat(tables, ignore_index = True)
collection = collection[
    ['AA_EXERCICIO',
     'CD_MUNICIPIO',
     'NM_MUNICIPIO',
     'NR_CNPJ_PRESTADOR_CONTA',
     'SG_PARTIDO',
     'DS_TP_ORIGEM_DOACAO',
     'NR_CPF_CNPJ_DOADOR',
     'CD_MUNICIPIO_DOADOR',
     'NM_MUNICIPIO_DOADOR',
     'DS_TP_ESPECIE_RECURSO',
     'VR_RECEITA']
]
collection = collection[collection['VR_RECEITA'] != '0']
collection.replace('#NULO#', np.nan, inplace = True)
collection['VR_RECEITA'] = collection['VR_RECEITA'].str.replace(',', '.').astype(float)

def map_ano_eleicao(ano):
    if ano in [2017, 2018]:
        return 1
    elif ano in [2019, 2020]:
        return 2
    elif ano in [2021, 2022]:
        return 3
    elif ano in [2023, 2024]:
        return 4
    else:
        return np.nan
    
collection['NR_ELEICAO'] = collection['AA_EXERCICIO'].apply(map_ano_eleicao)
collection.sample(10)

C:\Users\gusta\AppData\Local\Temp\ipykernel_9452\4087380051.py:16: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file, encoding = 'latin1', sep = ';')
C:\Users\gusta\AppData\Local\Temp\ipykernel_9452\4087380051.py:16: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file, encoding = 'latin1', sep = ';')
C:\Users\gusta\AppData\Local\Temp\ipykernel_9452\4087380051.py:16: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file, encoding = 'latin1', sep = ';')
C:\Users\gusta\AppData\Local\Temp\ipykernel_9452\4087380051.py:16: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file, encoding = 'latin1', sep = ';')
C:\Users\gusta\AppData\Local\Temp\ipykernel_9452\4087380051.py:16: DtypeWarning: Columns (5) hav

,AA_EXERCICIO,CD_MUNICIPIO,NM_MUNICIPIO,NR_CNPJ_PRESTADOR_CONTA,SG_PARTIDO,DS_TP_ORIGEM_DOACAO,NR_CPF_CNPJ_DOADOR,CD_MUNICIPIO_DOADOR,NM_MUNICIPIO_DOADOR,DS_TP_ESPECIE_RECURSO,VR_RECEITA,NR_ELEICAO
58900,2017,8693,CHARQUEADAS,3495008000164,PTB,Recursos de Pessoas Físicas,00970970048,8693,CHARQUEADAS,Depósito em espécie,29.00,1
428429,2022,8771,NOVO HAMBURGO,1351803000153,PSDB,Recursos de Pessoas Físicas,92331998000,8771,NOVO HAMBURGO,Transferência eletrônica,180.00,3
38553,2017,8511,ALVORADA,1402426000134,PSB,Recursos de Pessoas Físicas,38718456049,8511,ALVORADA,Depósito em espécie,100.00,1
189143,2019,8771,NOVO HAMBURGO,1351803000153,PSDB,Recursos de Pessoas Físicas,46984674034,8771,NOVO HAMBURGO,Transferência eletrônica,180.00,2
463593,2023,6083,CANDIOTA,92922145000168,MDB,Recursos de Pessoas Físicas,00009198024,6083,CANDIOTA,Transferência eletrônica,140.00,4
387352,2022,7303,VISTA GAÚCHA,94727195000110,MDB,Recursos de Pessoas Físicas,75038951015,7303,VISTA GAÚCHA,Aviso de Crédito,54.36,3
419322,2022,6015,SÃO JOSÉ DOS AUSENTES,10358846000151,PP,Recursos de Pessoas Físicas,56417837068,6015,SÃO JOSÉ DOS AUSENTES,Depósito em espécie,100.00,3
1789,2017,NaN,NaN,1343612000140,PCB,Recursos de Pessoas Físicas,01446445097,NaN,NaN,Depósito em espécie,50.00,1
515901,2023,8877,SÃO LEOPOLDO,93851491000165,PT,Recursos de Pessoas Físicas,00097904031,8877,SÃO LEOPOLDO,Transferência eletrônica,200.00,4
225279,2020,8759,NOVA ARAÇÁ,15355733000116,DEM,Recursos de Pessoas Físicas,48908436049,8759,NOVA ARAÇÁ,Transferência eletrônica,10.00,2


In [4]:
revenue = collection.groupby(['NR_ELEICAO', 'SG_PARTIDO']).agg(
    VR_RECEITA = ('VR_RECEITA', 'sum'),
    DS_TP_ESPECIE_RECURSO = ('DS_TP_ESPECIE_RECURSO', lambda x: x.mode()[0] if not x.mode().empty else None)
).reset_index()

revenue.to_csv('receita_geral_RS.csv', encoding = 'latin1', sep = ';', decimal = ',')

#### **Creating the "despesa_geral_RS.csv"**

In [5]:
files = [
    'despesa_anual_2017_RS.csv',
    'despesa_anual_2018_RS.csv',
    'despesa_anual_2019_RS.csv',
    'despesa_anual_2020_RS.csv',
    'despesa_anual_2021_RS.csv',
    'despesa_anual_2022_RS.csv',
    'despesa_anual_2023_RS.csv',
    'despesa_anual_2024_RS.csv',
]

tables = []
for file in files:
    data = pd.read_csv(file, encoding = 'latin1', sep = ';')
    tables.append(data)

collection = pd.concat(tables, ignore_index = True)
collection = collection[
    ['AA_EXERCICIO',
     'CD_MUNICIPIO',
     'NM_MUNICIPIO',
     'NR_CNPJ_PRESTADOR_CONTA',
     'SG_PARTIDO',
     'DS_TP_FORNECEDOR',
     'NR_CPF_CNPJ_FORNECEDOR',
     'DT_PAGAMENTO',
     'VR_GASTO']
]

def map_ano_eleicao(ano):
    if ano in [2017, 2018]:
        return 1
    elif ano in [2019, 2020]:
        return 2
    elif ano in [2021, 2022]:
        return 3
    elif ano in [2023, 2024]:
        return 4
    else:
        return np.nan
    
collection['NR_ELEICAO'] = collection['AA_EXERCICIO'].apply(map_ano_eleicao)

collection = collection[collection['VR_GASTO'] != '0']
collection.replace('#NULO#', np.nan, inplace = True)
collection['VR_GASTO'] = collection['VR_GASTO'].str.replace(',', '.').astype(float)
collection.sample(10)

C:\Users\gusta\AppData\Local\Temp\ipykernel_9452\3925497018.py:14: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file, encoding = 'latin1', sep = ';')
C:\Users\gusta\AppData\Local\Temp\ipykernel_9452\3925497018.py:14: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file, encoding = 'latin1', sep = ';')
C:\Users\gusta\AppData\Local\Temp\ipykernel_9452\3925497018.py:14: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file, encoding = 'latin1', sep = ';')
C:\Users\gusta\AppData\Local\Temp\ipykernel_9452\3925497018.py:14: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file, encoding = 'latin1', sep = ';')
C:\Users\gusta\AppData\Local\Temp\ipykernel_9452\3925497018.py:14: DtypeWarning:

,AA_EXERCICIO,CD_MUNICIPIO,NM_MUNICIPIO,NR_CNPJ_PRESTADOR_CONTA,SG_PARTIDO,DS_TP_FORNECEDOR,NR_CPF_CNPJ_FORNECEDOR,DT_PAGAMENTO,VR_GASTO,NR_ELEICAO
198683,2020,-1,NaN,8892050000132,PL,PESSOA JURÍDICA,191,01/08/2020,32.55,2
303868,2022,6039,MARATÁ,15515682000142,PSD,PESSOA JURÍDICA,92702067000196,02/06/2022,45.00,3
14425,2017,8801,PORTO ALEGRE,3195803000137,PT,PESSOA JURÍDICA,00655522000121,21/03/2017,17.40,1
361791,2023,-1,NaN,91340083000113,PT,PESSOA JURÍDICA,1082,23/08/2023,46.00,4
282717,2022,8669,GARIBALDI,1444072000190,PP,PESSOA JURÍDICA,92702067000196,02/02/2022,45.00,3
129938,2019,8627,DOM FELICIANO,4145153000188,PT,PESSOA JURÍDICA,00000000238090,05/11/2019,52.00,2
353986,2023,7391,JAQUIRANA,37269593000104,PSDB,PESSOA JURÍDICA,92702067000196,04/09/2023,59.90,4
357328,2023,-1,NaN,87582714000133,MDB,PESSOA JURÍDICA,02351877000152,05/05/2023,1410.47,4
271413,2021,6085,HULHA NEGRA,92922731000102,PT,PESSOA JURÍDICA,92702067058437,02/12/2021,45.00,3
19352,2017,8899,SAPIRANGA,3885983000189,PTB,PESSOA JURÍDICA,00360305000104,12/06/2017,35.00,1


In [6]:
expenses = collection.groupby(['NR_ELEICAO', 'SG_PARTIDO']).agg(
    VR_GASTO = ('VR_GASTO', 'sum'),
    DS_TP_FORNECEDOR = ('DS_TP_FORNECEDOR', lambda x: x.mode()[0] if not x.mode().empty else None)
).reset_index()

expenses.to_csv('despesa_geral_RS.csv', encoding = 'latin1', sep = ';', decimal = ',')

#### **Creating the "votacao_total_RS.csv"**

In [7]:
files = [
    'votacao_partido_munzona_2024_RS.csv',
    'votacao_partido_munzona_2022_RS.csv',
    'votacao_partido_munzona_2020_RS.csv',
    'votacao_partido_munzona_2018_RS.csv',
]

tables = []
for file in files:
    data = pd.read_csv(file, encoding = 'latin1', sep = ';')
    tables.append(data)

collection = pd.concat(tables, ignore_index = True)
collection = collection[
    ['ANO_ELEICAO',
     'NR_TURNO',
     'DS_ELEICAO',
     'NM_MUNICIPIO',
     'DS_CARGO',
     'SG_PARTIDO',
     'QT_VOTOS_NOMINAIS_VALIDOS']
]

def map_ano_eleicao(ano):
    if ano in [2017, 2018]:
        return 1
    elif ano in [2019, 2020]:
        return 2
    elif ano in [2021, 2022]:
        return 3
    elif ano in [2023, 2024]:
        return 4
    else:
        return np.nan

collection['NR_ELEICAO'] = collection['ANO_ELEICAO'].apply(map_ano_eleicao)
collection.replace('#NULO#', np.nan, inplace = True)
collection.sample(10)

,ANO_ELEICAO,NR_TURNO,DS_ELEICAO,NM_MUNICIPIO,DS_CARGO,SG_PARTIDO,QT_VOTOS_NOMINAIS_VALIDOS,NR_ELEICAO
38240,2022,1,Eleições Gerais Estaduais 2022,CAXIAS DO SUL,Governador,PL,42139,3
69107,2018,1,Eleições Gerais Estaduais 2018,VERA CRUZ,Deputado Federal,PRP,15,1
93230,2018,1,Eleições Gerais Estaduais 2018,BARRA DO QUARAÍ,Deputado Estadual,PROS,3,1
70013,2018,1,Eleições Gerais Estaduais 2018,VIADUTOS,Deputado Estadual,MDB,799,1
58709,2018,1,Eleições Gerais Estaduais 2018,TRÊS PASSOS,Deputado Federal,PSC,4,1
8253,2022,1,Eleições Gerais Estaduais 2022,MARATÁ,Deputado Estadual,DC,0,3
33158,2022,1,Eleições Gerais Estaduais 2022,ARROIO DO MEIO,Governador,PL,6005,3
9708,2022,1,Eleições Gerais Estaduais 2022,CAPÃO DO LEÃO,Governador,PDT,136,3
51382,2018,1,Eleições Gerais Estaduais 2018,IRAÍ,Deputado Estadual,PTC,0,1
79157,2018,1,Eleições Gerais Estaduais 2018,TRÊS ARROIOS,Deputado Estadual,PMN,4,1


In [8]:
voting = collection.groupby(['NR_TURNO', 'SG_PARTIDO', 'NR_ELEICAO']).agg(
    QT_VOTOS_NOMINAIS_VALIDOS = ('QT_VOTOS_NOMINAIS_VALIDOS', 'sum'),
).reset_index()

voting.to_csv('votacao_total_RS.csv', encoding = 'latin1', sep = ';', decimal = ',')

#### **Creating the "custo_voto_1T_RS_2018_2024.csv"**

In [9]:
expenses = pd.read_csv('despesa_geral_RS.csv', encoding = 'latin1', sep = ';', decimal = ',')
voting =  pd.read_csv('votacao_total_RS.csv', encoding = 'latin1', sep = ';', decimal = ',')

collection = pd.merge(voting, expenses, on = ['SG_PARTIDO', 'NR_ELEICAO'], how = 'left')

collection['GASTO_POR_VOTO'] = collection['VR_GASTO'] / collection['QT_VOTOS_NOMINAIS_VALIDOS']

collection = collection[
    ['NR_ELEICAO',
     'SG_PARTIDO',
     'NR_TURNO',
     'VR_GASTO',
     'QT_VOTOS_NOMINAIS_VALIDOS',
     'GASTO_POR_VOTO']
]

collection['NR_TURNO'] = collection['NR_TURNO'] == 1
walter_kannemann = collection[collection['NR_TURNO']]
walter_kannemann.sample(10)

,NR_ELEICAO,SG_PARTIDO,NR_TURNO,VR_GASTO,QT_VOTOS_NOMINAIS_VALIDOS,GASTO_POR_VOTO
43,4,PL,True,1855491.19,1452667,1.277300
96,1,PTB,True,6536740.84,650155,10.054127
12,4,DC,True,NaN,7593,NaN
48,2,PODE,True,1234039.39,53817,22.930289
57,1,PR,True,NaN,352605,NaN
115,4,SOLIDARIEDADE,True,NaN,24067,NaN
1,4,AGIR,True,NaN,8794,NaN
77,4,PSD,True,2235282.60,429804,5.200702
120,4,UP,True,67293.62,2911,23.117011
14,2,DEM,True,2599251.23,338816,7.671572


In [10]:
walter_kannemann.to_csv('custo_voto_1T_RS_2018_2024.csv', encoding = 'latin1', sep = ';', decimal=',')

#### **Creating the "correl_dataset.csv"**

In [11]:
expenses = pd.read_csv('despesa_geral_RS.csv', encoding='latin1', sep=';', decimal=',')
expenses = expenses[['SG_PARTIDO', 'NR_ELEICAO', 'VR_GASTO']]

revenues = pd.read_csv('receita_geral_RS.csv', encoding='latin1', sep=';', decimal=',')
revenues = revenues[['SG_PARTIDO', 'NR_ELEICAO', 'VR_RECEITA']]

collection = pd.merge(revenues, expenses, on=['SG_PARTIDO', 'NR_ELEICAO'], how='left')

correlations = {}

for party, group in collection.groupby('SG_PARTIDO'):
    correlation = group[['VR_RECEITA', 'VR_GASTO']].corr().iloc[0, 1]
    correlations[party] = correlation

correlations_df = pd.DataFrame(list(correlations.items()), columns=['SG_PARTIDO', 'CORRELACAO'])
correlations_df = correlations_df.dropna()

In [12]:
correlations_df.to_csv('correl_dataset.csv', encoding = 'latin1', sep = ';', decimal=',')
collection.to_csv('receitas_despesas.csv', encoding = 'latin1', sep = ';', decimal=',')

#### **Creating the "partidos_mais_votados.csv"**

In [ ]:
votes = pd.read_csv('custo_voto_1T_RS_2018_2024.csv', encoding='latin1', sep=';', decimal=',')

top5 = votes[['NR_ELEICAO', 'SG_PARTIDO', 'QT_VOTOS_NOMINAIS_VALIDOS']]

top5_sorted = top5.sort_values(by=['NR_ELEICAO', 'QT_VOTOS_NOMINAIS_VALIDOS'], ascending=[True, False])

top5_partidos = top5_sorted.groupby('NR_ELEICAO').head(5).reset_index(drop=True)

top5_partidos.to_csv('partidos_mais_votados.csv', encoding='latin1', sep=';', decimal=',')